<a href="https://colab.research.google.com/github/DiegoReyes123/Image-Classifier/blob/main/Image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision matplotlib



In [2]:
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)


In [3]:
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)   # input layer -> hidden
        self.fc2 = nn.Linear(128, 64)      # hidden -> hidden
        self.fc3 = nn.Linear(64, 10)       # hidden -> output (10 classes)

    def forward(self, x):
        # flatten 28x28 images into a 1D vector
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

# Create model instance
model = Classifier()
model


Classifier(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [4]:
import torch.optim as optim

# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

dataiter = iter(test_loader)
images, labels = next(dataiter)

# Pick first image
img = images[0]
plt.imshow(img.squeeze(), cmap='gray')

with torch.no_grad():
    logps = model(img.unsqueeze(0))
ps = torch.exp(logps)
print("Predicted class:", ps.argmax().item())


In [ ]:
# Save trained model weights to a file
torch.save(model.state_dict(), "fashion_mnist_fc.pth")
print("Saved: fashion_mnist_fc.pth")


In [ ]:
from google.colab import files
uploaded = files.upload()   # choose the image file you want to classify
img_name = next(iter(uploaded))   # filename string
print("Uploaded:", img_name)


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch

# Define same preprocessing used during training
preprocess = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),   # force 1 channel
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load and preprocess
img = Image.open(img_name).convert('RGB')   # open any RGB image safely
input_tensor = preprocess(img).unsqueeze(0)  # shape: (1, 1, 28, 28)

# For display, show the resized grayscale image
plt.figure(figsize=(3,3))
plt.imshow(img.convert('L').resize((140,140)), cmap='gray')
plt.axis('off')
plt.title("Input (resized to 28x28 for model)")
plt.show()


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
# FashionMNIST class names
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

# Move inputs + model to device
input_tensor = input_tensor.to(device)
model = model.to(device)

with torch.no_grad():
    logps = model(input_tensor)        # model returns log probabilities (log-softmax)
    ps = torch.exp(logps)              # convert to probabilities

topk = 3
top_p, top_class = ps.topk(topk, dim=1)

# Print top-3 predictions with percentages
for i in range(topk):
    cls_idx = top_class[0][i].item()
    prob = top_p[0][i].item() * 100
    print(f"Rank {i+1}: {classes[cls_idx]} ({prob:.1f}%)")


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
# Load a pretrained ResNet18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model = model.to(device)
model.eval()

# Get class labels
weights = models.ResNet18_Weights.DEFAULT
preprocess = weights.transforms()  # includes resize, crop, normalize
categories = weights.meta["categories"]


In [ ]:
from google.colab import files

uploaded = files.upload()  # choose a real image (jpg/png)
img_path = list(uploaded.keys())[0]
print("Image uploaded:", img_path)


In [ ]:
img = Image.open(img_path).convert("RGB")
input_tensor = preprocess(img).unsqueeze(0).to(device)


In [ ]:
with torch.no_grad():
    preds = model(input_tensor)

# Get top-5 predictions
topk = 5
probabilities = torch.nn.functional.softmax(preds[0], dim=0)
top_p, top_class = probabilities.topk(topk)

for i in range(topk):
    cls_idx = top_class[i].item()
    prob = top_p[i].item() * 100
    print(f"Rank {i+1}: {categories[cls_idx]} ({prob:.1f}%)")


In [ ]:
!pip install gradio

import gradio as gr
import torch
from PIL import Image

def predict_image(img):
    input_tensor = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)
    top_p, top_class = probs.topk(3)
    results = {categories[top_class[i].item()]: float(top_p[i].item()) for i in range(3)}
    return results

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Real-World Image Classifier",
    description="Upload any image and see top-3 predictions from a pretrained ResNet18 model."
)

iface.launch(share=True)


In [3]:
!pip install --upgrade gradio



In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import gradio as gr


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
# Load ResNet18 pretrained on ImageNet
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)
model = model.to(device)
model.eval()  # set to evaluation mode


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
preprocess = weights.transforms()       # includes resize, crop, normalize
categories = weights.meta["categories"]  # ImageNet labels


In [6]:
def predict_image(img):
    # Preprocess the uploaded image
    input_tensor = preprocess(img).unsqueeze(0).to(device)

    # Run the model
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)

    # Get top-3 predictions
    top_p, top_class = probs.topk(3)
    results = {categories[top_class[i].item()]: float(top_p[i].item()) for i in range(3)}
    return results


In [8]:
!pip install --upgrade gradio


In [9]:
# Imports
import torch
from torchvision import models
from PIL import Image
import gradio as gr

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet18
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights).to(device)
model.eval()

# Preprocessing and labels
preprocess = weights.transforms()
categories = weights.meta["categories"]

# Prediction function
def predict_image(img):
    input_tensor = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)
    top_p, top_class = probs.topk(3)
    return {categories[top_class[i].item()]: float(top_p[i].item()) for i in range(3)}


In [11]:
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # <-- removes source="webcam"
    outputs=gr.Label(num_top_classes=3),
    title="Real-Time Image Classifier",
    description="Upload an image to get top-3 predictions from the ResNet18 model."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c0fcd1849ac4ef5f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
